In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils import class_weight
from imblearn.keras import BalancedBatchGenerator


In [ ]:
# Configuration
CONFIG = {
    "data_paths": {
        "train": "data/train",
        "val": "data/validation",
        "test": "data/test"
    },
    "model_save_path": "models/emotion_model_v2.h5",
    "input_shape": (48, 48, 1),
    "batch_size": 64,
    "epochs": 100,
    "class_names": ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
}

In [ ]:
def handle_class_imbalance(generator):
    """Calculate class weights for imbalanced dataset"""
    class_weights = class_weight.compute_class_weight(
        'balanced',
        classes=np.unique(generator.classes),
        y=generator.classes
    )
    return dict(enumerate(class_weights))


In [ ]:
def create_data_generators():
    """Create augmented data generators with validation split"""
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=25,  # Increased for minority classes
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Add more augmentation for minority classes (especially Disgust)
    train_generator = train_datagen.flow_from_directory(
        CONFIG["data_paths"]["train"],
        target_size=CONFIG["input_shape"][:2],
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=CONFIG["batch_size"],
        shuffle=True
    )

    val_generator = val_test_datagen.flow_from_directory(
        CONFIG["data_paths"]["val"],
        target_size=CONFIG["input_shape"][:2],
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=CONFIG["batch_size"],
        shuffle=False
    )

    test_generator = val_test_datagen.flow_from_directory(
        CONFIG["data_paths"]["test"],
        target_size=CONFIG["input_shape"][:2],
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=CONFIG["batch_size"],
        shuffle=False
    )

    return train_generator, val_generator, test_generator

In [ ]:
def build_model():
    """Create CNN model with regularization"""
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=CONFIG["input_shape"]),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.3),

        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.4),

        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.5),

        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.6),
        Dense(7, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
def train():
    """Main training function"""
    # Create data generators
    train_gen, val_gen, test_gen = create_data_generators()
    
    # Handle class imbalance
    class_weights = handle_class_imbalance(train_gen)
    
    # Build model
    model = build_model()
    
    # Callbacks
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
        ModelCheckpoint(
            CONFIG["model_save_path"],
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]
    
    # Train with balanced batches
    history = model.fit(
        BalancedBatchGenerator(
            train_gen,
            train_gen.classes,
            batch_size=CONFIG["batch_size"],
            random_state=42
        ),
        steps_per_epoch=train_gen.samples // CONFIG["batch_size"],
        epochs=CONFIG["epochs"],
        validation_data=val_gen,
        class_weight=class_weights,
        callbacks=callbacks
    )
    
    # Final evaluation
    test_loss, test_acc = model.evaluate(test_gen)
    print(f"\nFinal Test Accuracy: {test_acc:.2%}")
    print(f"Final Test Loss: {test_loss:.4}")

In [ ]:
if __name__ == "__main__":
    # Create necessary directories
    os.makedirs(os.path.dirname(CONFIG["model_save_path"]), exist_ok=True)
    train()